# Распределение баллов

Задание 1: до **15** баллов

Задание 2: до **10** баллов

Задание 3: до **20** баллов

Задание 4: до **15** баллов

Задание 5: до **20** баллов


Дополнительно: до **20** баллов

Время выполнения всех заданий на GPU < 20мин.



# Вспомогательный код

Зафиксируем seeds для воспроизводимости результатов экспериментов.

In [ ]:
import numpy as np
import random
import torch


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


set_random_seed(42)

# Задание 1. Функция свертки

Реализуйте функцию свёртки `conv(x, kernel, pad)`, где:

* $\text{x}$ — двумерный массив размером $(W,H)$;
* $\text{kernel}$ — ядро свёртки размером $(K_w, K_h)$;
* $\text{pad}$ — ширина дополнения с каждой из сторон массива, $\text{pad} \geq 0$.

Шаг ядра свёртки $\text{stride}$ полагается равным единице

Результатом работы функции будет массив $\text{out}$, содержащий результат свертки входных данных с $\text{kernel}$, имеющий размер $(W', H')$:

- $W' = 1 + (W + 2 \cdot pad - K_w)\ $;
- $H' = 1 + (H + 2 \cdot pad - K_h)\ $.

Импорт необходимых библиотек:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data

Функция свертки:

In [ ]:
def conv(x, kernel, pad):
    # size
    h, w = x.shape
    k_h, k_w = kernel.shape

    # padding
    img = np.zeros((h + 2 * pad, w + 2 * pad))
    img[pad : pad + h, pad : pad + w] = x

    out = np.zeros((h + 2 * pad - k_h + 1, w + 2 * pad - k_w + 1))

    for i in range(0, h + 2 * pad - k_h + 1):
        for j in range(0, w + 2 * pad - k_w + 1):
            patch = img[i : i + k_h, j : j + k_w]
            out[i, j] = patch.flatten() @ kernel.flatten()

    return out

Тест

In [ ]:
# fmt: off
M = np.array([[0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0]])
# fmt: on

kernel = np.array([[1, 0, -1]])
actual = conv(M, kernel, pad=1)

print(actual.shape, "\n", actual)

# fmt: off
expected = np.array([[0, 0, 0, 0, 0],
                     [0, -1, 0, 1, 0],
                     [0, -1, 0, 1, 0],
                     [0, -1, 0, 1, 0],
                     [0, -1, 0, 1, 0],
                     [0, -1, 0, 1, 0],
                     [0, 0, 0, 0, 0]])
# fmt: on


assert np.array_equal(expected, actual), "Error"

Воспользуйтесь созданной функцией `conv` для применения [оператора Собеля](https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80_%D0%A1%D0%BE%D0%B1%D0%B5%D0%BB%D1%8F) к изображению.

Выведите на экран:
* изначальное изображение,
* приближённые производные по вертикали,
* приближённые производные по горизонтали,
* норму градиента (корень от суммы квадратов производных).

In [ ]:
camera = data.camera()
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].imshow(camera, cmap="gray", vmin=0, vmax=255)

sobel_x = np.array([[+1, 0, -1], [+2, 0, -2], [+1, 0, -1]])

sobel_y = sobel_x.T

gx = conv(camera, sobel_x, pad=1)
axs[0, 1].imshow(gx, cmap="gray", vmin=0, vmax=255)

gy = conv(camera, sobel_x.T, pad=1)
axs[1, 0].imshow(gy, cmap="gray", vmin=0, vmax=255)

g = np.sqrt(gx**2 + gy**2)
axs[1, 1].imshow(g, cmap="gray", vmin=0, vmax=255)

plt.show()

## Формат результата

Результатом являются 4 изображения, пример:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX06/result_1_task_ex06.png" width="500">


# Задание 2. Создание сверточной сети для  MNIST

Создайте сверточную сеть на PyTorch и обучите ее на MNIST.
* Используйте не более трёх [сверточных](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) и не более двух [полносвязных](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) слоев, а также слои [пулинга](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) (слой пулинга и аргумент `stride` у сверточных слоев помогут уменьшить число параметров линейного слоя).
* Не применяйте слои других типов.
* Функцию активации выберите на свое усмотрение.
* Рекомендуется использовать код для обучения из лекции №5.
* Отладку кода рекомендуем производить на небольшой части датасета (как обычно).

Импорт необходимых библиотек:

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.pooling import MaxPool2d
from torchvision import datasets, transforms, utils

Нейронная сеть:

In [ ]:
class MnistCNN(nn.Module):
    def __init__(self, input_shape=(1, 28, 28)):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 16, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
        )

        out = self.conv(torch.randn(input_shape).unsqueeze(0))

        self.fc = nn.Sequential(
            nn.Linear(int(out.shape[1]), 512), nn.ReLU(), nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        scores = self.fc(x)
        return scores

Функция для подсчета точности.
Изменять код в данном блоке не требуется.

In [ ]:
def get_correct_count(pred, labels):
    _, predicted = torch.max(pred.data, 1)
    return (predicted.cpu() == labels.cpu()).sum().item()


@torch.inference_mode()  # this annotation disable grad computation
def validate(model, test_loader, device="cpu"):
    correct, total = 0, 0
    for imgs, labels in test_loader:
        pred = model(imgs.to(device))
        total += labels.size(0)
        correct += get_correct_count(pred, labels)
    return correct / total

Загрузите [MNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST):

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.13), (0.3))]
)

mnist = datasets.MNIST(root="./", train=True, download=True, transform=transform)

(
    train_set,
    val_set,
) = torch.utils.data.random_split(
    mnist, [50000, 10000], generator=torch.Generator().manual_seed(42)
)

val_loader = torch.utils.data.DataLoader(val_set, batch_size=512, num_workers=2)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=512, num_workers=2)

Разместите код для обучения в этом блоке.

* Используйте GPU. Для этого Вам необходимо в верхней панели выбрать `Среда выполнения > Сменить среду выполнения` и заменить None на GPU.

* Для оценки точности используйте функцию `validate`.

* Подумайте над тем, чтобы поместить код для обучения в функцию или класс, который вы сможете использовать в следующих заданиях.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


def train(model, train_loader, val_loader, lr=0.01, epochs=10, title=""):
    global device
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)  # Weight update
    criterion = nn.CrossEntropyLoss()  # Loss function

    for epoch in range(epochs):
        loss_hist = []
        correct, total = 0, 0
        for imgs, labels in train_loader:
            optimizer.zero_grad()
            out = model(imgs.to(device))
            correct += get_correct_count(out.cpu(), labels)
            loss = criterion(out, labels.to(device))
            loss.backward()
            loss_hist.append(loss.item())
            optimizer.step()
            total += len(labels)
        print("Loss/train", np.mean(loss_hist))
        print("Accuracy/val", validate(model, val_loader, device=device))
        print("Accuracy/train", correct / total)

In [ ]:
%%time
model = MnistCNN()
model.to(device)
model.train()
train(model, train_loader, val_loader, lr=0.01, epochs=6)

Оценка результата на тестовой выборке:

In [ ]:
testset = datasets.MNIST(root="./", train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=True)
accuracy = validate(model, test_loader, device)

print(f"Accuracy on TEST {accuracy:.2f}")

## Формат результата

Результатом является сверточная сеть, обученная на MNIST, с точностью не ниже 0.9

# Задание 3. Переход на Lightning

При обучении моделей в PyTorch нам постоянно приходится переписывать цикл обучения (train loop). Это дублирование кода, которое нарушает принцип [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself).

Кроме того, нам нужно следить за процессом обучения модели. Например, если loss взрывается или выходит на плато, как правило, есть смысл остановить обучение. Чтобы контролировать этот процесс, приходится добавлять дополнительный код для вывода и/или логирования метрик.

При проведении реальных экспериментов логирование результатов станет необходимым. И в этом задании вы научитесь работать с фреймворком [Lightning](https://lightning.ai/), который облегчает написание train loop, логирование результатов, и.т.п.

Подзадачи:

1. Перепишите tain loop из задания №2 с использованием фреймворка [Lightning](https://lightning.ai/).
2. Выберите [один из инструментов для работы с логами](https://lightning.ai/docs/pytorch/stable/extensions/logging.html), который поддерживат Lightning, и инициализируйте его.
3. Обучите модель из второго задания. В процессе обучения сохраняйте в лог значения accuracy/train, accuracy/val и loss/train на каждой эпохе.
4. Выведите графики метрик, собранных во время выполнения п.3


Можно использовать пакет [torchmetrics](https://torchmetrics.readthedocs.io/en/stable/).

[Дополнительная информация про логирование метрик](https://lightning.ai/docs/pytorch/stable/extensions/logging.html#automatic-logging)

Установка и импорт необходимых библиотек

In [ ]:
!pip install -q lightning torchmetrics tbparse

In [ ]:
import torch
import torchmetrics
import lightning as L
import torch.nn as nn
import matplotlib.pyplot as plt

from torch import optim
from tbparse import SummaryReader
from torch.nn.modules.pooling import MaxPool2d
from torchvision import datasets, transforms, utils

### Запуск Tensorboard

По умолчанию Lightning иcпользует [Tensorboard](https://github.com/Gan4x4/ml_snippets/blob/main/Training/Tensorboard.ipynb)


In [ ]:
# colab magic command, run only once
%load_ext tensorboard

Попробуем его запустить

In [ ]:
# %reload_ext tensorboard

%tensorboard --logdir lightning_logs

Tensorboard не всегда корректно открывается в colab. Если вы видите в окне выше ошибку, используйте другой логгер, например [WandB](https://lightning.ai/docs/pytorch/stable/extensions/generated/lightning.pytorch.loggers.WandbLogger.html#lightning.pytorch.loggers.WandbLogger).

### Код LightningModule

Этим кодом мы заменим наш train loop из второго задания. Ваша задача — дописать недостающий код.

In [ ]:
import lightning as L
from torch import optim
import torchmetrics


class LitBasic(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.valid_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        self.train_acc.update(out, y)
        self.log("loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.valid_acc.update(out, y)

    def on_train_epoch_end(self):
        self.log("accuracy/train", self.train_acc.compute(), prog_bar=True)
        self.train_acc.reset()

    def on_validation_epoch_end(self):
        self.log("accuracy/val", self.valid_acc.compute(), prog_bar=True)
        self.valid_acc.reset()

### Подготовка датасета

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.13), (0.3))]
)

mnist = datasets.MNIST(root="./", train=True, download=True, transform=transform)

train_set, val_set, _ = torch.utils.data.random_split(mnist, [10000, 3000, 47000])

val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, num_workers=2)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, num_workers=2)

In [ ]:
L.seed_everything(42)
model = MnistCNN()
lit_model = LitBasic(model)
trainer = L.Trainer(max_epochs=10, log_every_n_steps=5)
trainer.fit(model=lit_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Полученные логи можно конвертировать в pandas при помощи [tbparse](https://github.com/j3soon/tbparse) и отобразить, используя matplotlib.

Чтобы получить таблицу, где названия метрик соответствуют названиям столбцов, используем параметр `pivot=True`.

In [ ]:
from tbparse import SummaryReader

log_dir = trainer.logger.experiment.get_logdir()  # lightning_logs/version_x

reader = SummaryReader(log_dir, pivot=True)
df = reader.scalars
df.head()

In [ ]:
plt.figure(figsize=(16, 3))


def drop_nan(df, tag):
    return df[~df[tag].isna()].loc[:, tag]


plt.subplot(1, 2, 1).set_title("Accuracy")

plt.plot(drop_nan(df, "accuracy/train"), label="train")
plt.plot(drop_nan(df, "accuracy/val"), label="val")
plt.legend(loc="lower right")

plt.subplot(1, 2, 2).set_title("Train loss")
plt.plot(drop_nan(df, "loss"))
plt.show()

## Формат результата

Результатом является график, отрисованный при помощи выбранного вами инструмента. Пример:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX06/result_3_task_ex06.png" width="800">


# Задание 4. Собственные аугментации

В этом задании предлагается применить к датасету CIFAR-10 аугментации и визуализировать получившиеся результаты. Требуется воспользоваться готовыми аугментациями из `torchvision.transforms`, а также реализовать собственный метод для аугментации изображений.

Импорт необходимых библиотек:

In [ ]:
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Функция для вывода изображений:

In [ ]:
def show_images(dataset, tag, img_count=32):
    """
    Helper method to display first img_count images from dataset
    """

    loader = DataLoader(dataset, batch_size=img_count)

    # Get first batch from dataloader
    dataiter = iter(loader)
    images, labels = next(dataiter)

    # create grid of images
    img_grid = torchvision.utils.make_grid(images)

    # plot
    plt.figure(figsize=(10, 7))
    plt.imshow(img_grid.permute((1, 2, 0)))
    plt.title(tag)
    plt.axis("off")
    plt.show()

Загрузите CIFAR-10 и:

* выберите два преобразования из [списка](https://pytorch.org/vision/stable/transforms.html),
* примените их к изображениям из датасета,
* выведите измененные изображения.

In [ ]:
testset = datasets.CIFAR10(
    "content",
    train=False,
    transform=transforms.Compose([transforms.ToTensor()]),
    download=True,
)

show_images(testset, "Without augmentation")

In [ ]:
base_augmentation = transforms.Compose(
    [
        # Your code here
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomErasing(),  # some augmentations work only with tensors
    ]
)

testset.transform = base_augmentation
show_images(testset, "Flip and cutout")

## Создание собственной аугментации
* Создайте собственную аугментацию, которой нет в `torchvision.transforms`. Например, сделайте изображение квадратным при помощи padding, или обнулите чать пикселей, или уменьшите изображение и затем восстановите его до прежнего размера ...

* Выведите измененные изображения. При использовании функции `show_images` обратите внимание, что список трансформаций должен содержать `transforms.ToTensor()`.

In [ ]:
# Your code here


class DropPixel:  # fill pixel with black color with probability p
    def __init__(self, p=0.1):
        self.p = p

    def __call__(self, pil):
        np_im = np.array(pil)
        # create random mask
        ind = np.random.choice(a=[True, False], size=pil.size, p=[self.p, 1 - self.p])
        np_im[ind] = [0, 0, 0]  # black
        return np_im


custom_augmentation = transforms.Compose(
    [
        # Your code here
        DropPixel(0.2),
        transforms.ToTensor(),
    ]
)

testset.transform = custom_augmentation
show_images(testset, "DropPixel")

## Формат результата

* без аугментаций,
* с двумя аугментациями из torchvision,
* со своей собственной аугментацией.

# Задание 5. Создание сверточной сети для CIFAR-10

Создайте сверточную сеть на PyTorch и обучите ее на СIFAR-10. Цель — получить лучшее качество, чем у полносвязной сети, которую мы обучали в 5-й лекции.

* Используйте не более трёх [сверточных](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) и не более двух [полносвязных](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) слоев, а также один слой [пулинга](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) между ними (слой пулинга и аргумент `stride` у сверточных слоев помогут избежать большой размерности на входе в линейный слой).
* Не применяйте слои других типов.
* Функцию активации выберите на свое усмотрение.
* Допустимо применить аугментации к входным данным.

Импорт необходимых библиотек:

In [ ]:
import torch
import torchmetrics
import torch.nn as nn
import lightning as L

from torch import optim
from torchsummary import summary
from torch.nn.modules.pooling import MaxPool2d
from torchvision import datasets, transforms, utils

Завершите реализацию модели

In [ ]:
class CifarCNN(nn.Module):
    def __init__(self, input_shape=(3, 32, 32)):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1, stride=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.Flatten(),
        )

        out = self.conv(torch.randn(input_shape).unsqueeze(0))

        self.fc = nn.Sequential(
            nn.Linear(int(out.shape[1]), 1024), nn.ReLU(), nn.Linear(1024, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        scores = self.fc(x)
        return scores

In [ ]:
from torchsummary import summary

model = CifarCNN()
model.cpu()
summary(model, (3, 32, 32), device="cpu")

Загрузите CIFAR-10:

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2434, 0.2615]),
        transforms.RandomErasing(),
    ]
)

cifar = datasets.CIFAR10(
    root="./", train=True, download=True, transform=train_transform
)

cifar_train_set, cifar_val_set = torch.utils.data.random_split(cifar, [45000, 5000])

cifar_train_loader = torch.utils.data.DataLoader(
    cifar_train_set, batch_size=256, shuffle=True, num_workers=2
)
cifar_val_loader = torch.utils.data.DataLoader(
    cifar_val_set, batch_size=256, num_workers=2
)

Разместите код для обучения в этом блоке.

* Используйте GPU. Для этого Вам необходимо в верхней панели выбрать `Среда выполнения > Сменить среду выполнения` и заменить `Hardware accelerator` на T4 GPU или другой доступный GPU ускоритель.

* Используйте Lightning. Рекомендуем унаследоваться от класса `LitBasic` из задания №3 и реализовать в классе-наследнике методы для проверки результатов на тестовом датасете. Можете использовать `self.valid_acc` или определить другой атрибут класса.

In [ ]:
class LitTest(LitBasic):
    def __init__(self, model):
        super().__init__(model)

    def test_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.valid_acc.update(out, y)

    def on_test_epoch_end(self):
        self.log("accuracy/test", self.valid_acc.compute(), prog_bar=True)
        self.valid_acc.reset()

In [ ]:
L.seed_everything(42)
cifar_model = CifarCNN()
cifar_lit_model = LitTest(cifar_model)
trainer = L.Trainer(max_epochs=25)

In [ ]:
trainer.fit(
    model=cifar_lit_model,
    train_dataloaders=cifar_train_loader,
    val_dataloaders=cifar_val_loader,
)

## Оценка результата на тестовой выборке

In [ ]:
test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2434, 0.2615]),
        # No augmentation on test
    ]
)

cifar_test = datasets.CIFAR10(
    root="./", train=False, download=True, transform=test_transform
)

test_loader = torch.utils.data.DataLoader(cifar_test, batch_size=256, num_workers=2)

In [ ]:
# Load the best checkpoint automatically (lightning tracks this for you)
trainer.test(
    model=cifar_lit_model, dataloaders=test_loader, verbose=True, ckpt_path="best"
)

## Формат результата

Результатом является сверточная сеть, обученная на CIFAR-10, с точностью не ниже 0.6